In [1]:
import pandas as pd
import numpy as np

# Telemetria

In [ ]:
df_telemetria = pd.read_csv('../dados/dados_totvs/telemetria_1.csv')

In [ ]:
for i in range(2, 12):
    proximo_df = pd.read_csv(f'../dados/dados_totvs/telemetria_{i}.csv')
    df_telemetria = pd.concat([df_telemetria, proximo_df], ignore_index=True)

In [9]:
# Conmvetendo o código da loja em código do cliente (removendo últiumos 2 dígitos)
df_telemetria['clienteid'] = df_telemetria['clienteid'].str[:-2]

In [10]:
# Definindo função de agregação pela moda

def agg_moda(x):
    m = x.dropna().mode()
    if not m.empty:
        return m.iloc[0]
    return np.nan

In [11]:
# Criando coluna para contagem de casos

df_telemetria['contagem'] = 1

In [12]:
# Agrupando o df por cliente

df_telemetria = df_telemetria.groupby('clienteid').agg({
                             'moduloid': agg_moda,
                             'productlineid': agg_moda,
                             'slotid': agg_moda,
                             'statuslicenca': agg_moda,
                             'contagem': 'count'}).reset_index()

In [13]:
df_telemetria.columns = ['CD_CLIENTE',
                       'telemetria_moduloid_moda',
                       'telemetria_productlineid_moda',
                       'telemetria_slotid_moda',
                       'telemetria_statuslicenca_moda',
                       'telemetria_contagem']

# Dados Clientes

In [ ]:
df_dados_clientes = pd.read_csv('../dados/dados_totvs/dados_clientes.csv', sep=';')

In [16]:
# Convertendo valores monetarios para floar/int

df_dados_clientes["VL_TOTAL_CONTRATO"] = df_dados_clientes["VL_TOTAL_CONTRATO"].str.replace(",", ".").astype(float)

In [17]:
# Transformando contratos de valor negativo em 0

df_dados_clientes.loc[df_dados_clientes["VL_TOTAL_CONTRATO"] < 0, "VL_TOTAL_CONTRATO"] = 0

In [18]:
# Preenchendo valores nulos do segmento do cliente como "Não Informado"

df_dados_clientes['DS_SUBSEGMENTO'] = df_dados_clientes['DS_SUBSEGMENTO'].fillna('NAO INFORMADO')

In [19]:
# Preenchendo valores nulos da amrca totvs como "Não Informado"

df_dados_clientes['MARCA_TOTVS'] = df_dados_clientes['MARCA_TOTVS'].fillna('NAO INFORMADO')

In [20]:
# Preenchendo valores nulos do modal comercial como "Não Informado"

df_dados_clientes['MODAL_COMERC'] = df_dados_clientes['MODAL_COMERC'].fillna('NAO INFORMADO')

In [21]:
# Preenchendo valores nulos da situação contratual como "Não Informado"

df_dados_clientes['SITUACAO_CONTRATO'] = df_dados_clientes['SITUACAO_CONTRATO'].fillna('NAO INFORMADO')

In [22]:
def agg_moda_faixa(x):
    # Conta as frequências
    contagem = x.dropna().value_counts()
    
    if contagem.empty:
        return np.nan
    
    # Pega o mais frequente
    mais_frequente = contagem.index[0]
    
    # Se for "Sem Informações de Faturamento" e existir outro valor
    if mais_frequente == "Sem Informações de Faturamento" and len(contagem) > 1:
        return contagem.index[1]  # segundo valor mais frequente
    
    return mais_frequente

In [23]:
# Criando o dataframe agruapdo

df_dados_clientes = df_dados_clientes.groupby('CD_CLIENTE').agg({'DS_PROD':agg_moda,
                             'DS_LIN_REC':agg_moda,
                             'CIDADE':agg_moda,
                             'DS_CNAE':agg_moda,
                             'DS_SEGMENTO':agg_moda,
                             'DS_SUBSEGMENTO':agg_moda,
                             'FAT_FAIXA':agg_moda_faixa,
                             'MARCA_TOTVS':agg_moda,
                             'MODAL_COMERC':agg_moda,
                             'PAIS':agg_moda,
                             'PERIODICIDADE':agg_moda,
                             'UF':agg_moda,
                             'VL_TOTAL_CONTRATO':'sum'}).reset_index()

In [24]:
# Renomeando colunas

df_dados_clientes.columns = ['CD_CLIENTE', 'dados_clientes_DS_PROD_moda', 'dados_clientes_DS_LIN_REC_moda', 'dados_clientes_CIDADE_moda',
                             'dados_clientes_DS_CNAE_moda',
       'dados_clientes_DS_SEGMENTO_moda', 'dados_clientes_DS_SUBSEGMENTO_moda', 'dados_clientes_FAT_FAIXA_moda',
       'dados_clientes_MARCA_TOTVS_moda',
       'dados_clientes_MODAL_COMERC_moda', 'dados_clientes_PAIS_moda', 'dados_clientes_PERIODICIDADE_moda', 'dados_clientes_UF_moda',
       'dados_clientes_VL_TOTAL_CONTRATO_soma']

In [25]:
# Dividindo os valores de contrato em faixas

# Definir o tamanho da faixa (1 milhão)
tamanho_faixa = 500_000

# Determinar valor máximo da coluna
max_valor = df_dados_clientes["dados_clientes_VL_TOTAL_CONTRATO_soma"].max()

# Criar limites automaticamente: de 0 até o próximo milhão acima do máximo
limites = np.arange(0, max_valor + tamanho_faixa, tamanho_faixa)

# Criar rótulos das faixas
labels = [f"{int(l/500_000)*0.5}-{int(limites[i+1]/500_000)*0.5}M" for i, l in enumerate(limites[:-1])]

# Criar a coluna com as faixas
df_dados_clientes["dados_clientes_faixa_VL_TOTAL_CONTRATO_soma"] = pd.cut(df_dados_clientes["dados_clientes_VL_TOTAL_CONTRATO_soma"], bins=limites, labels=labels, include_lowest=True)

# Clientes Desde

In [ ]:
df_clientes_desde = pd.read_csv('../dados/dados_totvs/clientes_desde.csv', sep=';')

In [27]:
# Converter string -> datetime
df_clientes_desde["CLIENTE_DESDE"] = pd.to_datetime(
    df_clientes_desde["CLIENTE_DESDE"], format="%Y-%m-%d", errors="coerce"
)

In [28]:
# Calcular diferença em dias até hoje
df_clientes_desde["CLIENTE_DESDE"] = (
    pd.Timestamp.today().normalize() - df_clientes_desde["CLIENTE_DESDE"]
).dt.days.astype("Int64")  # Int64 aceita nulos

In [29]:
# Renomeando coluna de cliente para fazer o merge
df_clientes_desde.rename(columns={"CLIENTE": "CD_CLIENTE"}, inplace=True)

In [33]:
df_clientes_desde.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10615 entries, 0 to 10614
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CD_CLIENTE     10615 non-null  object
 1   CLIENTE_DESDE  10615 non-null  Int64 
dtypes: Int64(1), object(1)
memory usage: 176.4+ KB


In [34]:
df_clientes_desde['CD_CLIENTE'].nunique()

10615

# Contratações Últimos 12 Meses

In [ ]:
df_contratacoes_ultimos_12_meses = pd.read_csv('../dados/dados_totvs/contratacoes_ultimos_12_meses.csv', sep=';')

In [31]:
# Convertendo valores monetarios para floar/int
df_contratacoes_ultimos_12_meses['VLR_CONTRATACOES_12M'] = df_contratacoes_ultimos_12_meses['VLR_CONTRATACOES_12M'].str.replace(",", ".").astype(float)

In [35]:
df_contratacoes_ultimos_12_meses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4314 entries, 0 to 4313
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CD_CLIENTE            4314 non-null   object 
 1   QTD_CONTRATACOES_12M  4314 non-null   int64  
 2   VLR_CONTRATACOES_12M  4314 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 101.2+ KB


In [36]:
df_contratacoes_ultimos_12_meses['CD_CLIENTE'].nunique()

4314

# MRR

In [ ]:
df_mrr = pd.read_csv('../dados/dados_totvs/mrr.csv', sep=';')

In [38]:
# Renomeando coluna de cliente para fazer o merge
df_mrr.rename(columns={"CLIENTE": "CD_CLIENTE"}, inplace=True)

In [39]:
df_mrr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7309 entries, 0 to 7308
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CD_CLIENTE  7309 non-null   object 
 1   MRR_12M     7309 non-null   float64
dtypes: float64(1), object(1)
memory usage: 114.3+ KB


In [40]:
df_mrr['CD_CLIENTE'].nunique()

7309

# NPS Relacional

In [ ]:
df_nps_relacional = pd.read_csv('../dados/dados_totvs/nps_relacional.csv', sep=';')

In [42]:
df_nps_relacional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14143 entries, 0 to 14142
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   respondedAt                14143 non-null  object 
 1   metadata_codcliente        14143 non-null  object 
 2   resposta_NPS               14143 non-null  int64  
 3   resposta_unidade           11311 non-null  float64
 4   Nota_SupTec_Agilidade      8067 non-null   float64
 5   Nota_SupTec_Atendimento    7161 non-null   float64
 6   Nota_Comercial             6907 non-null   float64
 7   Nota_Custos                7504 non-null   float64
 8   Nota_AdmFin_Atendimento    6494 non-null   float64
 9   Nota_Software              5821 non-null   float64
 10  Nota_Software_Atualizacao  7435 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 1.2+ MB


In [44]:
# Renomeando coluna de cliente para fazer o merge
df_nps_relacional.rename(columns={"metadata_codcliente": "CD_CLIENTE"}, inplace=True)

In [45]:
df_nps_relacional['CD_CLIENTE'].nunique()

4862

In [46]:
# Removendo data de resposta (sem importancia para a clusterização)
df_nps_relacional = df_nps_relacional.drop('respondedAt', axis=1)

In [47]:
# agrupando por cliente e pegando a mediana (mais robusta que a média)
df_nps_relacional = df_nps_relacional.groupby('CD_CLIENTE').agg('median').reset_index()

In [48]:
df_nps_relacional.columns

Index(['CD_CLIENTE', 'resposta_NPS', 'resposta_unidade',
       'Nota_SupTec_Agilidade', 'Nota_SupTec_Atendimento', 'Nota_Comercial',
       'Nota_Custos', 'Nota_AdmFin_Atendimento', 'Nota_Software',
       'Nota_Software_Atualizacao'],
      dtype='object')

In [49]:
df_nps_relacional.columns = ['CD_CLIENTE', 'nps_relacional_resposta_NPS', 'nps_relacional_resposta_unidade',
       'nps_relacional_Nota_SupTec_Agilidade', 'nps_relacional_Nota_SupTec_Atendimento', 'nps_relacional_Nota_Comercial',
       'nps_relacional_Nota_Custos', 'nps_relacional_Nota_AdmFin_Atendimento', 'nps_relacional_Nota_Software',
       'nps_relacional_Nota_Software_Atualizacao']

In [77]:
# Preenchendo os valores nulos com a mediana da resposta NPS

colunas_para_preencher = df_nps_relacional.columns.drop(['CD_CLIENTE', 'nps_relacional_resposta_NPS']).tolist()

for coluna in colunas_para_preencher:
    df_nps_relacional[coluna] = df_nps_relacional[coluna].fillna(df_nps_relacional["nps_relacional_resposta_NPS"])

# Histórico

In [ ]:
df_historico = pd.read_csv('../dados/dados_totvs/historico.csv', sep=';')

In [65]:
df_historico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22740 entries, 0 to 22739
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   NR_PROPOSTA             22740 non-null  object
 1   ITEM_PROPOSTA           22740 non-null  int64 
 2   DT_UPLOAD               22740 non-null  object
 3   HOSPEDAGEM              22740 non-null  object
 4   CD_CLI                  22740 non-null  object
 5   FAT_FAIXA               22740 non-null  object
 6   CD_PROD                 22740 non-null  object
 7   QTD                     22740 non-null  object
 8   MESES_BONIF             22740 non-null  int64 
 9   VL_PCT_DESC_TEMP        22740 non-null  object
 10  VL_PCT_DESCONTO         22740 non-null  object
 11  PRC_UNITARIO            22740 non-null  object
 12  VL_DESCONTO_TEMPORARIO  22740 non-null  object
 13  VL_TOTAL                22740 non-null  object
 14  VL_FULL                 22740 non-null  object
 15  VL

In [66]:
df_historico.columns

Index(['NR_PROPOSTA', 'ITEM_PROPOSTA', 'DT_UPLOAD', 'HOSPEDAGEM', 'CD_CLI',
       'FAT_FAIXA', 'CD_PROD', 'QTD', 'MESES_BONIF', 'VL_PCT_DESC_TEMP',
       'VL_PCT_DESCONTO', 'PRC_UNITARIO', 'VL_DESCONTO_TEMPORARIO', 'VL_TOTAL',
       'VL_FULL', 'VL_DESCONTO'],
      dtype='object')

In [67]:
# faça o mesmo para as colunas de valores monetários, se necessário
cols_numericas = [
    'QTD', 'VL_PCT_DESC_TEMP',
       'VL_PCT_DESCONTO', 'PRC_UNITARIO', 'VL_DESCONTO_TEMPORARIO', 'VL_TOTAL',
       'VL_FULL', 'VL_DESCONTO'
]

for col in cols_numericas:
    df_historico[col] = pd.to_numeric(df_historico[col], errors='coerce')

In [69]:
df_historico = df_historico.groupby('CD_CLI').agg({
    'NR_PROPOSTA': 'count',
    'ITEM_PROPOSTA': agg_moda,
    'HOSPEDAGEM': agg_moda,
    'CD_PROD': agg_moda,
    'QTD': 'mean',
    'MESES_BONIF': agg_moda,
    'VL_PCT_DESC_TEMP': ['median', 'sum'],
    'VL_PCT_DESCONTO': ['median', 'sum'],
    'PRC_UNITARIO': ['median', 'sum'],
    'VL_DESCONTO_TEMPORARIO': ['median', 'sum'],
    'VL_TOTAL': ['median', 'sum'],
    'VL_FULL': ['median', 'sum'],
    'VL_DESCONTO': ['median', 'sum'],
}).reset_index()

In [70]:
df_historico.columns

MultiIndex([(                'CD_CLI',         ''),
            (           'NR_PROPOSTA',    'count'),
            (         'ITEM_PROPOSTA', 'agg_moda'),
            (            'HOSPEDAGEM', 'agg_moda'),
            (               'CD_PROD', 'agg_moda'),
            (                   'QTD',     'mean'),
            (           'MESES_BONIF', 'agg_moda'),
            (      'VL_PCT_DESC_TEMP',   'median'),
            (      'VL_PCT_DESC_TEMP',      'sum'),
            (       'VL_PCT_DESCONTO',   'median'),
            (       'VL_PCT_DESCONTO',      'sum'),
            (          'PRC_UNITARIO',   'median'),
            (          'PRC_UNITARIO',      'sum'),
            ('VL_DESCONTO_TEMPORARIO',   'median'),
            ('VL_DESCONTO_TEMPORARIO',      'sum'),
            (              'VL_TOTAL',   'median'),
            (              'VL_TOTAL',      'sum'),
            (               'VL_FULL',   'median'),
            (               'VL_FULL',      'sum'),
            

In [71]:
df_historico.columns = [
 'CD_CLIENTE',
 'historico_NR_PROPOSTA_count',
 'historico_ITEM_PROPOSTA_agg_moda',
 'historico_HOSPEDAGEM_agg_moda',
 'historico_CD_PROD_agg_moda',
 'historico_QTD_mean',
 'historico_MESES_BONIF_agg_moda',
 'historico_VL_PCT_DESC_TEMP_median',
 'historico_VL_PCT_DESC_TEMP_sum',
 'historico_VL_PCT_DESCONTO_median',
 'historico_VL_PCT_DESCONTO_sum',
 'historico_PRC_UNITARIO_median',
 'historico_PRC_UNITARIO_sum',
 'historico_VL_DESCONTO_TEMPORARIO_median',
 'historico_VL_DESCONTO_TEMPORARIO_sum',
 'historico_VL_TOTAL_median',
 'historico_VL_TOTAL_sum',
 'historico_VL_FULL_median',
 'historico_VL_FULL_sum',
 'historico_VL_DESCONTO_median',
 'historico_VL_DESCONTO_sum'
]


# NPS Transacional

In [ ]:
df_nps_transacional_aquisicao = pd.read_csv('../dados/dados_totvs/nps_transacional_aquisicao.csv', sep=';', encoding='latin1')

In [100]:
df_nps_transacional_aquisicao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 7 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   CD_CLIENTE                                     165 non-null    object 
 1   nps_transacional_aquisicao_Nota_NPS            165 non-null    float64
 2   nps_transacional_aquisicao_Nota_Agilidade      165 non-null    float64
 3   nps_transacional_aquisicao_Nota_Conhecimento   165 non-null    float64
 4   nps_transacional_aquisicao_Nota_Custo          165 non-null    float64
 5   nps_transacional_aquisicao_Nota_Facilidade     165 non-null    float64
 6   nps_transacional_aquisicao_Nota_Flexibilidade  165 non-null    float64
dtypes: float64(6), object(1)
memory usage: 9.2+ KB


In [95]:
df_nps_transacional_implantacao = pd.read_csv('nps_transacional_implantacao.csv', sep=';', encoding='latin1')

In [99]:
df_nps_transacional_implantacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662 entries, 0 to 661
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cód. Cliente       661 non-null    object 
 1   Data da Resposta   662 non-null    object 
 2   Nota NPS           662 non-null    int64  
 3   Nota Metodologia   417 non-null    float64
 4   Nota Gestao        417 non-null    float64
 5   Nota Conhecimento  417 non-null    float64
 6   Nota Qualidade     417 non-null    float64
 7   Nota Comunicacao   417 non-null    float64
 8   Nota Prazos        417 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 46.7+ KB


In [ ]:
df_nps_transacional_onboarding = pd.read_csv('../dados/dados_totvs/nps_transacional_onboarding.csv', sep=';', encoding='latin1')

In [98]:
df_nps_transacional_onboarding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 8 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                           --------------  -----  
 0   Data de resposta                                                                                                                 208 non-null    object 
 1   Cod Cliente                                                                                                                      207 non-null    object 
 2   Em uma escala de 0 a 10, quanto você recomenda o Onboarding da TOTVS para um amigo ou colega?.                                   208 non-null    int64  
 3   Em uma escala de 0 a 10, o quanto você acredita que o atendimento CS Onboarding ajudou no início da s

In [ ]:
# Dataset relacional já basta

# Merge

In [101]:
df = pd.merge(
    df_telemetria, 
    df_dados_clientes, 
    on="CD_CLIENTE",
    how="outer"
)

In [102]:
df = pd.merge(
    df, 
    df_clientes_desde, 
    on="CD_CLIENTE",
    how="outer"
)

In [103]:
df = pd.merge(
    df, 
    df_contratacoes_ultimos_12_meses, 
    on="CD_CLIENTE",
    how="outer"
)

In [104]:
df = pd.merge(
    df, 
    df_mrr, 
    on="CD_CLIENTE",
    how="outer"
)

In [105]:
df = pd.merge(
    df, 
    df_nps_relacional, 
    on="CD_CLIENTE",
    how="outer"
)

In [106]:
df = pd.merge(
    df, 
    df_historico, 
    on="CD_CLIENTE",
    how="outer"
)

# Análise

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10616 entries, 0 to 10615
Data columns (total 53 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   CD_CLIENTE                                   10616 non-null  object  
 1   telemetria_moduloid_moda                     4465 non-null   float64 
 2   telemetria_productlineid_moda                4465 non-null   float64 
 3   telemetria_slotid_moda                       4465 non-null   float64 
 4   telemetria_statuslicenca_moda                4393 non-null   object  
 5   telemetria_contagem                          4465 non-null   float64 
 6   dados_clientes_DS_PROD_moda                  10615 non-null  object  
 7   dados_clientes_DS_LIN_REC_moda               10615 non-null  object  
 8   dados_clientes_CIDADE_moda                   10615 non-null  object  
 9   dados_clientes_DS_CNAE_moda                  10615 non-null  

In [108]:
# nulos
df.isnull().sum()/len(df) * 100

CD_CLIENTE                                       0.000000
telemetria_moduloid_moda                        57.940844
telemetria_productlineid_moda                   57.940844
telemetria_slotid_moda                          57.940844
telemetria_statuslicenca_moda                   58.619066
telemetria_contagem                             57.940844
dados_clientes_DS_PROD_moda                      0.009420
dados_clientes_DS_LIN_REC_moda                   0.009420
dados_clientes_CIDADE_moda                       0.009420
dados_clientes_DS_CNAE_moda                      0.009420
dados_clientes_DS_SEGMENTO_moda                  0.009420
dados_clientes_DS_SUBSEGMENTO_moda               0.009420
dados_clientes_FAT_FAIXA_moda                    0.009420
dados_clientes_MARCA_TOTVS_moda                  0.009420
dados_clientes_MODAL_COMERC_moda                 0.009420
dados_clientes_PAIS_moda                         0.009420
dados_clientes_PERIODICIDADE_moda                0.009420
dados_clientes